In [20]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as transforms
from imageio import mimwrite
from PIL import Image, ImageDraw
from sklearn.preprocessing import minmax_scale
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_video, write_video

from models import EchoNet

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
output_dir = 'output'
video_path = '/home/tienyu/data/EchoNet-Dynamic/Videos/'

In [22]:
model = EchoNet(device).to(device)
model.load_state_dict(torch.load('checkpoints/best.pt'))

<All keys matched successfully>

In [23]:
filelist = pd.read_csv('/home/tienyu/data/EchoNet-Dynamic/FileList.csv')
test_filelist = filelist[filelist.Split == 'TEST']

In [24]:
idx = 160
instance_filename = test_filelist.FileName.iloc[idx]
instance_fps = test_filelist.FPS.iloc[idx]
instance_path = os.path.join(video_path, instance_filename) + '.avi'

In [25]:
video = read_video(instance_path, pts_unit='sec')[0].permute(0, 3, 1, 2)/255.
grayscaled = transforms.Grayscale(num_output_channels=1)(video)
grayscaled = grayscaled.to(device)
model.eval()

masks = []
volumes = []
with torch.no_grad():
    for vs in DataLoader(grayscaled, batch_size=32):
        m, x = model(vs)
        masks.append(m)
        volumes.append(x)
masks = torch.cat(masks).repeat(1, 3, 1, 1).detach().cpu()
masks = (torch.sigmoid(masks) > 0.5).float()
masks[:,:2,:,:] = 0 # make blue color mask
video = video.detach().cpu()
masked = torch.clip(masks + video, min=0., max=0.99)
volumes = torch.cat(volumes).detach().cpu().numpy().flatten()

In [26]:
frames = minmax_scale(np.arange(len(video)), feature_range=(5,100))
scaled = minmax_scale(-volumes, feature_range=(10,30))
output = []
for i, m in enumerate([transforms.ToPILImage()(m) for m in masked]):
    canvas = Image.new('RGB', (144, 144))
    canvas.paste(m, (0,0))
    draw = ImageDraw.Draw(canvas)
    annot = list(zip(frames[:i], scaled[:i]+110))
    draw.line(annot, joint='curve', fill='green')
    output.append(canvas)

In [14]:
mimwrite(os.path.join(output_dir, instance_filename)+'.avi', np.stack(output), fps=instance_fps.astype(float))
print(f"Output video is saved to {os.path.join(output_dir, instance_filename)+'.avi'}")

Output video is saved to output/0X20C87FB09C5485FA.avi


In [36]:
test_filelist.EF.iloc[idx]

55.12381681

In [51]:
s = 70
model._get_pseudo_ef(grayscaled[s:s+64:4].unsqueeze(0))*100

tensor([[54.5355]], device='cuda:0')

In [61]:
x = torch.rand(5,1)

In [66]:
j = torch.where(torch.tensor([0,1,0,1,1])!=0)
x[j]

tensor([[0.9939],
        [0.9980],
        [0.8540]])